<a href="https://colab.research.google.com/github/sudarshan-koirala/youtube-stuffs/blob/main/openai_assistant_firstlook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install the OpenAI Python library

In [1]:
%%capture
!pip install --upgrade openai watermark

In [2]:
#!pip list

In [3]:
!pip list | grep openai

openai                           1.1.1


In [4]:
%load_ext watermark
%watermark -a "Sudarshan Koirala" -vmp openai

Author: Sudarshan Koirala

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

openai: 1.1.1

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 5.15.120+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



## Setup Your API key

In [5]:

# get your openai api key from https://platform.openai.com/account/api-keys 🔑
import openai
import os
from getpass import getpass

OPENAI_API_KEY = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

··········


In [6]:
from openai import OpenAI

client = OpenAI()

## Create an assistant

In [7]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview"
)

## create a Thread
- Recommendation is to create one thread per user.
- Create messages and pass user-specific context and files in this thread.

In [8]:
thread = client.beta.threads.create()

## Add message to a thread

In [9]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [10]:
message

ThreadMessage(id='msg_T0dgrPNN9rJ1OmtXYtJfjN48', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1699364317, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_o5CnqCoHOmouqYCjtChYLhJp')

## Run the Assistant
- For the Assistant to respond to the user message, you need to create a Run. This makes the Assistant read the Thread and decide whether to call tools or simply use the model to best answer the user query.

In [11]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Sudarshan Koirala."
)

## Retrieve the assistant response

In [12]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [13]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [14]:
messages

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_T0dgrPNN9rJ1OmtXYtJfjN48', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1699364317, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_o5CnqCoHOmouqYCjtChYLhJp')], object='list', first_id='msg_T0dgrPNN9rJ1OmtXYtJfjN48', last_id='msg_T0dgrPNN9rJ1OmtXYtJfjN48', has_more=False)

In [15]:
# Assuming you have the data in a variable named 'messages'
# Access the data list of ThreadMessages
data = messages.data

if data:
    # Extract the first ThreadMessage
    first_thread_message = data[0]

    # Access the content of the first ThreadMessage
    content = first_thread_message.content

    if content:
        # Access the text from the content
        text = content[0].text

        # Now, the 'text' variable contains the text from the first ThreadMessage
        print(text)
    else:
        print("No content in the first ThreadMessage.")
else:
    print("No data available.")


Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?')


In [16]:
run

Run(id='run_70MjRCU8mljdTBWSzRPWMPqK', assistant_id='asst_fMua5Gz2FzofMLT41a1yoQhj', cancelled_at=None, completed_at=None, created_at=1699364318, expires_at=1699364918, failed_at=None, file_ids=[], instructions='Please address the user as Sudarshan Koirala.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_o5CnqCoHOmouqYCjtChYLhJp', tools=[ToolAssistantToolsCode(type='code_interpreter')])

In [17]:
run.status

'queued'

In [18]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [19]:
run.status

'completed'

In [20]:
run

Run(id='run_70MjRCU8mljdTBWSzRPWMPqK', assistant_id='asst_fMua5Gz2FzofMLT41a1yoQhj', cancelled_at=None, completed_at=1699364323, created_at=1699364318, expires_at=None, failed_at=None, file_ids=[], instructions='Please address the user as Sudarshan Koirala.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1699364318, status='completed', thread_id='thread_o5CnqCoHOmouqYCjtChYLhJp', tools=[ToolAssistantToolsCode(type='code_interpreter')])

In [33]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [34]:
messages

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_yuaUln4PJnoAeMIfhTL0Rt4O', assistant_id='asst_fMua5Gz2FzofMLT41a1yoQhj', content=[MessageContentText(text=Text(annotations=[], value='The solution to the equation \\(3x + 11 = 14\\) is \\(x = 1\\).'), type='text')], created_at=1699364323, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_70MjRCU8mljdTBWSzRPWMPqK', thread_id='thread_o5CnqCoHOmouqYCjtChYLhJp'), ThreadMessage(id='msg_T0dgrPNN9rJ1OmtXYtJfjN48', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1699364317, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_o5CnqCoHOmouqYCjtChYLhJp')], object='list', first_id='msg_yuaUln4PJnoAeMIfhTL0Rt4O', last_id='msg_T0dgrPNN9rJ1OmtXYtJfjN48', has_more=False)

In [35]:
# Assuming you have the data in a variable named 'messages'
# Access the data list of ThreadMessages
data = messages.data

if data:
    # Extract the first ThreadMessage
    first_thread_message = data[0]

    # Access the content of the first ThreadMessage
    content = first_thread_message.content

    if content:
        # Access the text from the content
        text = content[0].text

        # Now, the 'text' variable contains the text from the first ThreadMessage
        print(text)
    else:
        print("No content in the first ThreadMessage.")
else:
    print("No data available.")


Text(annotations=[], value='The solution to the equation \\(3x + 11 = 14\\) is \\(x = 1\\).')
